In [7]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [8]:
from threading import Thread

class StreamingThread(Thread):
    def __init__(self, ssc):
        Thread.__init__(self)
        self.ssc = ssc
    def run(self):
        ssc.start()
        ssc.awaitTermination()
    def stop(self):
        print('----- Stopping... this may take a few seconds -----')
        self.ssc.stop(stopSparkContext=False, stopGraceFully=True)

In [9]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [10]:
spark

In [11]:
from pyspark.streaming import StreamingContext
from pyspark.sql import Row
from pyspark.sql.functions import udf, struct, array, col, lit
from pyspark.sql.types import StringType
from pyspark.ml import Pipeline
from pyspark.ml import PipelineModel
from pyspark.sql.types import StructType, StructField, StringType, LongType
import pyspark.sql.functions as f
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import ltrim
from pyspark.ml.feature import Tokenizer
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.feature import CountVectorizer, StringIndexer

In [ ]:
#Converting all letters to lowercase
#df = df.withColumn("tweet_text",f.lower(f.col("tweet_text")))

#removing punctuations, numbers, http and spaces
#df =df.withColumn("tweet_text",f.regexp_replace(f.col("tweet_text"),'([^ a-zA-Z\'])',''))
#df = df.withColumn("tweet_text",f.regexp_replace(f.col("tweet_text"),'http.*?\\b',' '))
#df = df.withColumn("tweet_text",f.ltrim(f.regexp_replace(f.col("tweet_text"),'[\r\n\t\f\v ]+', ' ')))


#from pyspark.ml.feature import Tokenizer
#from pyspark.ml.feature import StopWordsRemover

#Splitting words

#tokenizer = Tokenizer(inputCol="tweet_text", outputCol="words")
#dataset = tokenizer.transform(df)

#pandas_df = dataset.select("*").toPandas()
#pandas_df['lemmatized'] = pandas_df['words'].apply(lambda lst:[lemmatizer.lemmatize(word) for word in lst])
#pandas_df['lemmatized']
#dataset2 = spark.createDataFrame(pandas_df)

#Vectorizing
#from pyspark.ml.feature import CountVectorizer, StringIndexer


#cv = CountVectorizer(inputCol="filtered", outputCol="features")
#label_stringIdx = StringIndexer(inputCol = "label", outputCol = "labelIndex")

#pipeline = Pipeline(stages=[cv, label_stringIdx])

#pipelineFit = pipeline.fit(dataset2)
#dataset2 = pipelineFit.transform(dataset2)

In [12]:
globals()['models_loaded'] = True
globals()['my_model'] = PipelineModel.load("lr_model")

def process(time, rdd):
    if rdd.isEmpty():
        return
    
    print("========= %s =========" % str(time))
    
    # Convert to data frame
    df = spark.read.json(rdd)
    df.show()
    
    # Utilize our predict function
   #df_withpreds = df.withColumn("pred", predict_udf(
       #struct([df[x] for x in df.columns])
    #))
    #df_withpreds.show()
        
    # Normally, you wouldn't use a UDF (User Defined Function) Python function to predict as we did here (you can)
    # but an MLlib model you've built and saved with Spark
    # In this case, you need to prevent loading your model in every call to "process" as follows:
    
    # Load in the model if not yet loaded:
    if not globals()['models_loaded']:
        # load in your models here
        globals()['my_model'] = '***' # Replace '***' with:    [...].load('my_logistic_regression')
        globals()['models_loaded'] = True
        
    prediction = globals()['my_model'].transform(df)
    selected = prediction.select("labelIndex", "prediction")
    for row in selected.collect():
        print(row)

In [13]:
ssc = StreamingContext(sc, 10)

In [14]:
lines = ssc.socketTextStream("seppe.net", 7778)
lines.foreachRDD(process)

In [15]:
ssc_t = StreamingThread(ssc)
ssc_t.start()

========= 2021-05-28 11:51:30 =========
+------+-------------------+--------------------+
| label|           tweet_id|          tweet_text|
+------+-------------------+--------------------+
|#china|1398212783217414149|#███████ have bee...|
+------+-------------------+--------------------+

Row(labelIndex=2.0, prediction=2.0)
========= 2021-05-28 11:51:40 =========
+------+-------------------+--------------------+
| label|           tweet_id|          tweet_text|
+------+-------------------+--------------------+
|#china|1398212667110744065|Dujiangyan Zhongs...|
|#china|1398212624253345792|Will add: There a...|
+------+-------------------+--------------------+

Row(labelIndex=2.0, prediction=1.0)
Row(labelIndex=2.0, prediction=2.0)
========= 2021-05-28 11:52:00 =========
+------+-------------------+--------------------+
| label|           tweet_id|          tweet_text|
+------+-------------------+--------------------+
|#china|1398212514781958148|Understanding #██...|
|#china|139821250995

In [16]:
ssc_t.stop()

----- Stopping... this may take a few seconds -----
========= 2021-05-28 11:52:50 =========
+------+-------------------+--------------------+
| label|           tweet_id|          tweet_text|
+------+-------------------+--------------------+
|#covid|1398212978000809985|Verified...
Oxyge...|
+------+-------------------+--------------------+

Row(labelIndex=0.0, prediction=0.0)
